Taller 2.2 - Concurrencia

Daniel Alberto González Pabón - C.C 1128462077

In [ ]:
import asyncio
import time
import aiohttp

In [ ]:
random_image_urls = [
    "https://picsum.photos/600/400?random=1",
    "https://picsum.photos/600/400?random=2",
    "https://picsum.photos/600/400?random=3",
    "https://picsum.photos/600/400?random=4"
]

unplash_image_urls = [
    "https://images.unsplash.com/photo-1506905925346-21bda4d32df4?w=800",
    "https://images.unsplash.com/photo-1441974231531-c6227db76b6e?w=800",
    "https://images.unsplash.com/photo-1518837695005-2083093ee35b?w=800"
]

json_data_urls = [
    "https://jsonplaceholder.typicode.com/posts/1",
    "https://jsonplaceholder.typicode.com/users/1"
]

csv_data_urls = [
    "https://raw.githubusercontent.com/cs109/2014_data/master/countries.csv",
    "https://raw.githubusercontent.com/plotly/datasets/master/iris.csv",
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv"
    ]

text_urls = [
    "https://www.gutenberg.org/files/11/11-0.txt",
    "https://raw.githubusercontent.com/python/cpython/main/README.rst",
    "https://raw.githubusercontent.com/microsoft/vscode/main/README.md",
    "https://raw.githubusercontent.com/tensorflow/tensorflow/master/README.md"
]

pdf_urls = [
    "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf",
    "https://www.africau.edu/images/default/sample.pdf"
    ]

audio_urls = [
    "https://www.soundjay.com/misc/sounds/bell-ringing-05.wav"
                ]

code_urls = [
    "https://raw.githubusercontent.com/django/django/main/setup.cfg",
    "https://raw.githubusercontent.com/pallets/flask/main/pyproject.toml",
    "https://raw.githubusercontent.com/numpy/numpy/main/requirements.txt",
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/setup.py"
    ]

mixed_urls = random_image_urls + unplash_image_urls + json_data_urls + csv_data_urls + text_urls + pdf_urls + audio_urls + code_urls

1) Método Asincronico

In [ ]:
async def download_site(session, url):
    async with session.get(url) as response:
        size = response.content.read()
        if size is None:
            data = await response.read()
            size = len(data)
        print("Read {0} from {1}".format(response.content_length, url))
        

async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.create_task(download_site(session, url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)
        

async def main():
    sites = [*mixed_urls]
    start_time = time.time()
    await download_all_sites(sites) #Agregué await para esperar a que se complete la tarea, con asyncio se retornaba un Runtime Error
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration} seconds")

In [ ]:
await main()

2) Método Threading

In [ ]:
import concurrent.futures
import requests
import threading

In [ ]:
thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def Descargar_Sitio(url):
    session = get_session()
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def Descargar_Todos_los_sitios(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(Descargar_Sitio, sites)

In [ ]:
Descargar_Todos_los_sitios(mixed_urls)

3) Método multiprocessing

In [ ]:
import multiprocessing

In [ ]:
session = None

In [106]:
def set_global_session():
    global session
    if not session:
        session = requests.Session()
        

def Descargar_Sitio_MP(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        print(f"{name}:Read {len(response.content)} from {url}")
        
def Descargar_Todos_los_sitios_MP(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)
        
def main():
    sites = [*mixed_urls]
    start_time = time.time()
    Descargar_Todos_los_sitios_MP(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")
    
